In [ ]:
svtype <- 'DEL'
sample <- 'HTZ-SV'
truth_set_file <- '../../../data/test/test.bedpe'
regions_for_filtering <- 'ENCFF001TDO.bed'
out_dir <- 'results'

In [ ]:
# SV callsets
callsets <- list()
callsets[['CNN']]  <- 'CNN_predictions.bedpe'
callsets[['GRIDSS']] <- '../../../data/test/vcf/gridss_out/gridss.vcf'
callsets[['Manta']] <- '../../../data/test/vcf/manta_out/manta.vcf'
callsets[['Lumpy']] <- '../../../data/test/vcf/lumpy_out/lumpy.vcf'
callsets[['DELLY']] <- '../../../data/test/vcf/delly_out/delly.vcf'

Load the [StructuralVariantAnnotation](https://bioconductor.org/packages/devel/bioc/vignettes/StructuralVariantAnnotation/inst/doc/vignettes.html) package

In [ ]:
suppressPackageStartupMessages(require(StructuralVariantAnnotation))
suppressPackageStartupMessages(require(dplyr))
suppressPackageStartupMessages(require(ggplot2))

[SV type inference](https://github.com/PapenfussLab/gridss/blob/7b1fedfed32af9e03ed5c6863d368a821a4c699f/example/simple-event-annotation.R#L9)

In [ ]:
infer_svtype <- function(gr)
{
  gr$svtype <-
    ifelse(
      seqnames(gr) != seqnames(partner(gr)),
      "TRA", # Using TRA instead of ITX or BP
      ifelse(
        gr$insLen >= abs(gr$svLen) * 0.7,
        "INS",
        ifelse(
          strand(gr) == strand(partner(gr)),
          "INV",
          ifelse(xor(
            start(gr) < start(partner(gr)), strand(gr) == "-"
          ), "DEL",
          "DUP")
        )
      )
    )
  return(gr)
}

In [ ]:
load_bed <- function(bed_file)
{
    bed_regions <- rtracklayer::import(bed_file)
    # set NCBI seqlevels
    seqlevelsStyle(bed_regions) <- "NCBI"
    return(bed_regions)
}

In [ ]:
load_bedpe <- function(bedpe_file, filter_regions)
{
  bedpe_gr <- pairs2breakpointgr(rtracklayer::import(bedpe_file))
  bedpe_gr <- filter_regions(bedpe_gr, load_bed(filter_regions), mode='remove')
  return(bedpe_gr)
}

In [ ]:
load_vcf <- function(vcf_file, svtype, caller, filter_regions)
{
    # Load VCF file
    vcf_gr <-
      VariantAnnotation::readVcf(vcf_file)
    
    # set NCBI seqlevels
    seqlevelsStyle(vcf_gr) <- 'NCBI'
    
    if(caller=='survivor')
    {
        # SURVIVOR simSV assigns LowQual to all artificial SVs
        vcf_gr <- vcf_gr[rowRanges(vcf_gr)$FILTER%in%c("LowQual")]
    }else{
        # Keep only SVs that passed the filtering (PASS or .)
        vcf_gr <- vcf_gr[rowRanges(vcf_gr)$FILTER%in%c("PASS",".")]
    }

    if (caller == 'lumpy')
    {
      # Read evidence support as a proxy for QUAL
      support <- unlist(info(vcf_gr)$SU)
      fixed(vcf_gr)$QUAL <- support
    } else if (caller == 'delly')
    {
      # Split-read support plus Paired-end read support as a proxy for QUAL
      sr_support <- info(vcf_gr)$SR
      sr_support[is.na(vcf_gr)] <- 0
      fixed(vcf_gr)$QUAL <-
        sr_support + info(vcf_gr)$PE
    }
    
    vcf_gr <- breakpointRanges(vcf_gr)
    vcf_gr <- infer_svtype(vcf_gr)
    
    # Select only one SV type
    vcf_gr <- vcf_gr[which(vcf_gr$svtype == svtype)]

    # Select SVs >= 50 bp
    vcf_gr <- vcf_gr[abs(vcf_gr$svLen) >= 50]
    
    #Filter regions
    vcf_gr <- filter_regions(vcf_gr, load_bed(filter_regions), mode='remove')
}

In [ ]:
filter_regions <- function(regions_to_filter, ref_regions, mode='remove')
{
    print(length(regions_to_filter))
  if (mode == 'keep')
  {
    result <- regions_to_filter[overlapsAny(regions_to_filter, ref_regions) &
         overlapsAny(partner(regions_to_filter), ref_regions), ]
  } else{
    result <- regions_to_filter[!(
      overlapsAny(regions_to_filter, ref_regions) |
        overlapsAny(partner(regions_to_filter), ref_regions)
    ), ]
  }
    print(length(result))
    return(result)
}

Load the truth set

In [ ]:
truth_set <- load_bedpe(truth_set_file, regions_for_filtering)
truth_set <- truth_set[truth_set$sourceId==svtype]
length(truth_set)

Load the SV callsets

In [ ]:
sv_regions <- list()
# sv_regions[['CNN']] <- load_bedpe(callsets[['CNN']], regions_for_filtering)
sv_regions[['GRIDSS']] <- load_vcf(callsets[['GRIDSS']], svtype, 'gridss', regions_for_filtering)
sv_regions[['Manta']] <- load_vcf(callsets[['Manta']], svtype, 'manta', regions_for_filtering)
sv_regions[['Lumpy']] <- load_vcf(callsets[['Lumpy']], svtype, 'lumpy', regions_for_filtering)
sv_regions[['DELLY']] <- load_vcf(callsets[['DELLY']], svtype, 'delly', regions_for_filtering)

Add SV caller name

In [ ]:
for (c in names(sv_regions))
{
  sv_regions[[c]]$caller <- c
}

Compute overlap

In [ ]:
for (c in names(sv_regions))
{
    
  sv_regions[[c]]$truth_matches <-
    countBreakpointOverlaps(
      sv_regions[[c]],
      truth_set,
      # read pair based callers make imprecise calls.
      # A margin around the call position is required when matching with the truth set
      maxgap = 200,
      # Since we added a maxgap, we also need to restrict the mismatch between the
      # size of the events. We don't want to match a 100bp deletion with a
      # 5bp duplication. This will happen if we have a 100bp margin but don't also
      # require an approximate size match as well
      sizemargin = 0.25,
      ignore.strand = TRUE,
      # We also don't want to match a 20bp deletion with a 20bp deletion 80bp away
      # by restricting the margin based on the size of the event, we can make sure
      # that simple events actually do overlap
      restrictMarginToSizeMultiple = 0.5,
      # Some callers make duplicate calls and will sometimes report a variant multiple
      # times with slightly different bounds. countOnlyBest prevents these being
      # double-counted as multiple true positives.
      countOnlyBest = TRUE
    )

}
    
sv_regions <- unlist(GRangesList(sv_regions))

In [ ]:
sv_regions

Plotting Precision and Recall as in [StructuralVariantAnnotation vignette](https://bioconductor.org/packages/devel/bioc/vignettes/StructuralVariantAnnotation/inst/doc/vignettes.html)

In [ ]:
main.title <- c("SURVIVOR truth set")
names(main.title) <- c(sample)

ggplot(
  as.data.frame(sv_regions) %>%
    dplyr::select(QUAL, caller, truth_matches) %>%
    dplyr::group_by(caller, QUAL) %>%
    dplyr::summarise(calls = n(),
                     tp = sum(truth_matches > 0)) %>%
    dplyr::group_by(caller) %>%
    dplyr::arrange(dplyr::desc(QUAL)) %>%
    dplyr::mutate(
      cum_tp = cumsum(tp),
      cum_n = cumsum(calls),
      cum_fp = cum_n - cum_tp,
      precision = cum_tp / cum_n,
      recall = cum_tp / length(truth_set)
    )
) +
  aes(x = recall,
      y = precision,
      colour = caller) +
  geom_point() +
  geom_line() +
  scale_y_continuous(labels = scales::percent) +
  scale_x_continuous(
    labels = scales::percent,
    sec.axis = sec_axis( ~ (.) * length(truth_set), name = "true positives")
  ) +
  labs(title = main.title[sample])

In [ ]:
make_percent <- function(x){
  signif(x*100,digits = 4)
}

Summarize results

In [ ]:
res.df <- as.data.frame(sv_regions) %>%
    dplyr::select(caller, truth_matches) %>%
    dplyr::group_by(caller) %>%
    dplyr::summarise(calls = n(),
                   tp = sum(truth_matches > 0)) %>%
    dplyr::group_by(caller) %>%
    dplyr::mutate(
    cum_tp = cumsum(tp),
    cum_n = cumsum(calls),
    cum_fp = cum_n - cum_tp,
    precision = signif(cum_tp / cum_n, digits = 4),
    recall = signif(cum_tp / length(truth_svgr), digits = 4)
    )
res.df$F1 = with(res.df, 2 * (precision * recall) / (precision + recall))
res.df$precision <- make_percent(res.df$precision)
res.df$recall <- make_percent(res.df$recall)
res.df$F1 <- make_percent(res.df$F1)

In [ ]:
res.df